In [1]:
#Bounced Orders
import csv
import psycopg2
import pandas as pd
import numpy as np
import paramiko
from sshtunnel import SSHTunnelForwarder
# EC2 instance connection details
ec2_host = '65.1.183.184'
ec2_username = 'ubuntu'
ec2_pem_key_path = "C:/Users/kumar.rohit.KUMARROHIT-LPT/Desktop/emami-prod.pem"
# AWS RDS PostgreSQL database connection details
rds_host = 'emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com'
rds_port = 5432
rds_database = 'efrprod'
rds_user = 'emamireaduser'
rds_password = 'emamireadaccess'
try:
    # Create an SSH tunnel to the EC2 instance
    with SSHTunnelForwarder(
        (ec2_host, 22),
        ssh_username=ec2_username,
        ssh_pkey=ec2_pem_key_path,
        remote_bind_address=(rds_host, rds_port),
        local_bind_address=('localhost', 22)
    ) as tunnel:
        # Establish a connection to the RDS PostgreSQL database
        connection = psycopg2.connect(
            host='localhost',
            port=tunnel.local_bind_port,
            database=rds_database,
            user=rds_user,
            password=rds_password
        )
        cursor = connection.cursor()

        # Perform database operations here
        cursor.execute(
'''
SELECT 
Bounce_tbl.order_id,
Bounce_tbl.status,
Bounce_tbl.sku,
Bounce_tbl.name,
ord_qty.order_qty,
Bounce_tbl.deficit_quantity,
Bounce_tbl.Level_1_store,
Bounce_tbl.pincode,
Bounce_tbl.confirmed_on
FROM 
(with cte_bounce_orders as (
SELECT 
    o.order_id,
    CASE
		WHEN orders.state =0 THEN 'cart'
		WHEN orders.state =1 THEN 'pre_checkout'
		WHEN orders.state =2 THEN 'Checkout'
		WHEN orders.state =6 THEN 'Delivered'
		WHEN orders.state =7 THEN 'Cancelled'
		WHEN orders.state =14  THEN 'Return_completed'
		ELSE 'In-Progress'
		END AS status,
    v.sku,
    v.name,
    o.deficit_quantity,
    w.store_name AS Level_1_store,
    ar.pincode,
    ols.state_changed_on + INTERVAL '5 hours 30 minutes' AS confirmed_on,
	CONCAT (o.order_id,v.sku) as order_fr_code,
    CASE
	WHEN orders.customer_id IS NULL AND ols.channel = 1 THEN 'Mobile'
	WHEN ols.channel = 0 AND COALESCE(orders.delivery_remarks, '') LIKE '%FRSAATH%' THEN 'Saathi'
	WHEN ols.channel = 0 THEN 'Call Center'
	WHEN ols.channel = 1 THEN 'Saathi App'
	ELSE 'Website'
END as channel
FROM 
    order_deficit_quantity_logs o
LEFT JOIN 
    warehouses w ON o.warehouse_id = w.id
LEFT JOIN 
    areas ar ON o.area_id = ar.id
LEFT JOIN 
    variants v ON v.id = o.variant_id
LEFT JOIN 
    orders ON orders.id = o.order_id
LEFT JOIN 
    order_status_logs ols ON ols.order_id = o.order_id
WHERE
    ols.to_state = 3
    AND ols.state_changed_on > '2024-11-30 18:30:00'
    AND orders.state > 2
    AND ols.state_changed_on + INTERVAL '5 hours 30 minutes' > '2024-11-30 18:30:00'
)
select * From cte_bounce_orders where confirmed_on > '2024-11-30 18:30:00') Bounce_tbl

left join 

(
SELECT 
CONCAT (order_id,'FR-',variant_id) as order_fr_code,
quantity as order_qty
FROM line_items where order_id in 
(
SELECT
order_id
FROM order_status_logs
WHERE state_changed_on >'2024-11-30 18:30' AND to_state = 3
)) ord_qty
on ord_qty.order_fr_code = Bounce_tbl.order_fr_code
''')
        records = cursor.fetchall()
        # Write records to a CSV file
        with open('bounced_orders.csv', 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([desc[0] for desc in cursor.description])  # Write column headers
            writer.writerows(records)
        print("Data fetched and saved to bounced_orders.csv")
except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL:", error)
finally:
    # Close the database connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")


import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

def send_email(sender_email, sender_password, recipient_emails, cc_emails, subject, message, attachment_filename):
    # Construct the email body with signature
    body_with_signature = f"{message}\n\nThanks & Regards,\nKumar Rohit\nAnalytics Manager | Emami Frank Ross Limited\n8882041300\nwww.frankrossphramacy.com"
    
    # Set up the MIME
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = ', '.join(recipient_emails)
    msg['Cc'] = ', '.join(cc_emails) if cc_emails else None
    msg['Subject'] = subject
    
    # Add message body
    msg.attach(MIMEText(body_with_signature, 'plain'))
    
    # Attach file
    with open(attachment_filename, "rb") as attachment:
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())
    
    encoders.encode_base64(part)
    part.add_header(
        "Content-Disposition",
        f"attachment; filename= {attachment_filename}",
    )
    msg.attach(part)

    # Connect to the SMTP server
    with smtplib.SMTP('smtp-mail.outlook.com', 587) as server:
        server.starttls()  # Secure the connection
        server.login(sender_email, sender_password)  # Login to your email account
        # Combine the recipients and send the email
        recipients = recipient_emails + (cc_emails if cc_emails else [])
        server.sendmail(sender_email, recipients, msg.as_string())  # Send the email
        print("Email sent successfully")

# Set your email credentials and email details
sender_email = 'kumar.rohit@frankrosspharmacy.com'
sender_password = 'Outl00k@0125' #'Outl00k@1224'
recipient_emails = ['debasish.das@frankrosspharmacy.com']  # List of primary recipients
cc_emails = ['saurabh@frankrosspharmacy.com','sarit@frankrosspharmacy.com',
'nitin.vyas@frankrosspharmacy.com','avijit.b@frankrosspharmacy.com','debopam.dey@frankrosspharmacy.com',
'chiradeep.roygupta@frankrosspharmacy.com', 'saurabh@frankrosspharmacy.com','ajatia@frankrosspharmacy.com',
             'debtanu.baidya@frankrosspharmacy.com'] 
subject = 'Bounced Article List'
message = 'Hi Debasish,\n\nPFA excel file containing list of Bounced articles MTD'
attachment_filename = 'bounced_orders.csv'  # Filename of the CSV file

# Call the function to send the email with attachment
send_email(sender_email, sender_password, recipient_emails, cc_emails, subject, message, attachment_filename)
print("Updated at", pd.Timestamp.now())

C:\Users\kumar.rohit.KUMARROHIT-LPT\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


Data fetched and saved to bounced_orders.csv
PostgreSQL connection is closed
Email sent successfully
Updated at 2025-02-07 15:47:40.426941


In [2]:
import csv
import psycopg2
import pandas as pd
import numpy as np
import paramiko
from sshtunnel import SSHTunnelForwarder
# EC2 instance connection details
ec2_host = '65.1.183.184'
ec2_username = 'ubuntu'
ec2_pem_key_path = "C:/Users/kumar.rohit.KUMARROHIT-LPT/Desktop/emami-prod.pem"
# AWS RDS PostgreSQL database connection details
rds_host = 'emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com'
rds_port = 5432
rds_database = 'efrprod'
rds_user = 'emamireaduser'
rds_password = 'emamireadaccess'
try:
    # Create an SSH tunnel to the EC2 instance
    with SSHTunnelForwarder(
        (ec2_host, 22),
        ssh_username=ec2_username,
        ssh_pkey=ec2_pem_key_path,
        remote_bind_address=(rds_host, rds_port),
        local_bind_address=('localhost', 22)
    ) as tunnel:
        # Establish a connection to the RDS PostgreSQL database
        connection = psycopg2.connect(
            host='localhost',
            port=tunnel.local_bind_port,
            database=rds_database,
            user=rds_user,
            password=rds_password
        )
        cursor = connection.cursor()

        # Perform database operations here
        cursor.execute(
'''
with cte_bounce_orders as (
SELECT 
    o.order_id,
    orders.state,
    v.sku,
    v.name,
    o.deficit_quantity,
    w.store_name AS Level_1_store,
    ar.pincode,
    ols.state_changed_on + INTERVAL '5 hours 30 minutes' AS confirmed_on,
    CASE
        WHEN ols.channel = '0' THEN 'call center'
        WHEN ols.channel = '1' THEN 'mobile'
        ELSE 'website'
    END AS channel
FROM 
    order_deficit_quantity_logs o
LEFT JOIN 
    warehouses w ON o.warehouse_id = w.id
LEFT JOIN 
    areas ar ON o.area_id = ar.id
LEFT JOIN 
    variants v ON v.id = o.variant_id
LEFT JOIN 
    orders ON orders.id = o.order_id
LEFT JOIN 
    order_status_logs ols ON ols.order_id = o.order_id
WHERE
    ols.to_state = 3
    AND ols.state_changed_on > '2024-10-31 18:30:00'
    AND orders.state > 2
    AND ols.state_changed_on + INTERVAL '5 hours 30 minutes' > '2024-10-31 18:30:00'
)
select * From cte_bounce_orders where confirmed_on > '2024-10-31 18:30:00'
''')
        records = cursor.fetchall()
        # Write records to a CSV file
        with open('bounced_orders.csv', 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([desc[0] for desc in cursor.description])  # Write column headers
            writer.writerows(records)
        print("Data fetched and saved to bounced_orders.csv")
except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL:", error)
finally:
    # Close the database connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")


import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

def send_email(sender_email, sender_password, recipient_emails, cc_emails, subject, message, attachment_filename):
    # Construct the email body with signature
    body_with_signature = f"{message}\n\nThanks & Regards,\nKumar Rohit\nAnalytics Manager | Emami Frank Ross Limited\n8882041300\nwww.frankrossphramacy.com"
    
    # Set up the MIME
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = ', '.join(recipient_emails)
    msg['Cc'] = ', '.join(cc_emails) if cc_emails else None
    msg['Subject'] = subject
    
    # Add message body
    msg.attach(MIMEText(body_with_signature, 'plain'))
    
    # Attach file
    with open(attachment_filename, "rb") as attachment:
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())
    
    encoders.encode_base64(part)
    part.add_header(
        "Content-Disposition",
        f"attachment; filename= {attachment_filename}",
    )
    msg.attach(part)

    # Connect to the SMTP server
    with smtplib.SMTP('smtp-mail.outlook.com', 587) as server:
        server.starttls()  # Secure the connection
        server.login(sender_email, sender_password)  # Login to your email account
        # Combine the recipients and send the email
        recipients = recipient_emails + (cc_emails if cc_emails else [])
        server.sendmail(sender_email, recipients, msg.as_string())  # Send the email
        print("Email sent successfully")

# Set your email credentials and email details
sender_email = 'kumar.rohit@frankrosspharmacy.com'
sender_password = 'Outl00k@1124'
recipient_emails = ['sarit@frankrosspharmacy.com','debasish.das@frankrosspharmacy.com']  # List of primary recipients
cc_emails = ['saurabh@frankrosspharmacy.com',
'nitin.vyas@frankrosspharmacy.com','avijit.b@frankrosspharmacy.com','debopam.dey@frankrosspharmacy.com',
'chiradeep.roygupta@frankrosspharmacy.com', 'saurabh@frankrosspharmacy.com','ajatia@frankrosspharmacy.com',
             'debtanu.baidya@frankrosspharmacy.com'] 
subject = 'Bounced Article List'
message = 'Hi Sarit,\n\nPFA excel file containing list of Bounced articles MTD'
attachment_filename = 'bounced_orders.csv'  # Filename of the CSV file

# Call the function to send the email with attachment
send_email(sender_email, sender_password, recipient_emails, cc_emails, subject, message, attachment_filename)
print("Updated at", pd.Timestamp.now())

Data fetched and saved to bounced_orders.csv
PostgreSQL connection is closed
Email sent successfully
Updated at 2024-11-11 11:58:05.805348


# Stock Sync Report - Item Wise

In [3]:
import csv
import psycopg2
import pandas as pd
import numpy as np
import paramiko
from sshtunnel import SSHTunnelForwarder
# EC2 instance connection details
ec2_host = '65.1.183.184'
ec2_username = 'ubuntu'
ec2_pem_key_path = "C:/Users/kumar.rohit.KUMARROHIT-LPT/Desktop/emami-prod.pem"
# AWS RDS PostgreSQL database connection details
rds_host = 'emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com'
rds_port = 5432
rds_database = 'efrprod'
rds_user = 'emamireaduser'
rds_password = 'emamireadaccess'
try:
    # Create an SSH tunnel to the EC2 instance
    with SSHTunnelForwarder(
        (ec2_host, 22),
        ssh_username=ec2_username,
        ssh_pkey=ec2_pem_key_path,
        remote_bind_address=(rds_host, rds_port),
        local_bind_address=('localhost', 22)
    ) as tunnel:
        # Establish a connection to the RDS PostgreSQL database
        connection = psycopg2.connect(
            host='localhost',
            port=tunnel.local_bind_port,
            database=rds_database,
            user=rds_user,
            password=rds_password
        )
        cursor = connection.cursor()

        # Perform database operations here
        cursor.execute(
'''
SELECT variant_id,warehouse_id,available_quantity,last_synced_at FROM warehouse_variants where warehouse_id = 425
''')
        records = cursor.fetchall()
        # Write records to a CSV file
        with open('TWH_stock_sync.csv', 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([desc[0] for desc in cursor.description])  # Write column headers
            writer.writerows(records)
        print("Data fetched and saved to TWH_stock_sync.csv")
except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL:", error)
finally:
    # Close the database connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")


import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

def send_email(sender_email, sender_password, recipient_emails, cc_emails, subject, message, attachment_filename):
    # Construct the email body with signature
    body_with_signature = f"{message}\n\nThanks & Regards,\nKumar Rohit\nAnalytics Manager | Emami Frank Ross Limited\n8882041300\nwww.frankrossphramacy.com"
    
    # Set up the MIME
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = ', '.join(recipient_emails)
    msg['Cc'] = ', '.join(cc_emails) if cc_emails else None
    msg['Subject'] = subject
    
    # Add message body
    msg.attach(MIMEText(body_with_signature, 'plain'))
    
    # Attach file
    with open(attachment_filename, "rb") as attachment:
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())
    
    encoders.encode_base64(part)
    part.add_header(
        "Content-Disposition",
        f"attachment; filename= {attachment_filename}",
    )
    msg.attach(part)

    # Connect to the SMTP server
    with smtplib.SMTP('smtp-mail.outlook.com', 587) as server:
        server.starttls()  # Secure the connection
        server.login(sender_email, sender_password)  # Login to your email account
        # Combine the recipients and send the email
        recipients = recipient_emails + (cc_emails if cc_emails else [])
        server.sendmail(sender_email, recipients, msg.as_string())  # Send the email
        print("Email sent successfully")

# Set your email credentials and email details
sender_email = 'kumar.rohit@frankrosspharmacy.com'
sender_password = 'P@ssw0rd@0424'
recipient_emails = ['sarit@frankrosspharmacy.com','debasish.das@frankrosspharmacy.com']  # List of primary recipients
cc_emails = ['saurabh@frankrosspharmacy.com',
'nitin.vyas@frankrosspharmacy.com','avijit.b@frankrosspharmacy.com',
'chiradeep.roygupta@frankrosspharmacy.com', 'saurabh@frankrosspharmacy.com','ajatia@frankrosspharmacy.com'] 
subject = 'TWH_stock_sync_report'
message = 'Hi Sarit,\n\nPFA TWH_stock_sync report item wise'
attachment_filename = 'TWH_stock_sync.csv'  # Filename of the CSV file

# Call the function to send the email with attachment
send_email(sender_email, sender_password, recipient_emails, cc_emails, subject, message, attachment_filename)
print("Updated at", pd.Timestamp.now())

Data fetched and saved to TWH_stock_sync.csv
PostgreSQL connection is closed
Email sent successfully
Updated at 2024-05-03 11:38:30.946510
